One-sample z-test for proportions:
Apakah lebih dari 50% responden menggunakan Mobile Banking?

In [16]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

# Load data
df = pd.read_excel("Dataset_Tubes.xlsx")

# Kolom penggunaan Mobile Banking
mobile_col = 'Bagaimana_frekuensi_penggunaan_layanan_layanan_Bank_berikut___Mobile_Banking'
df_mb = df.dropna(subset=[mobile_col]).copy()
df_mb[mobile_col] = df_mb[mobile_col].str.strip().str.lower()

# Definisikan kategori pengguna aktif
aktif_keywords = [
    '2-5 kali per bulan',
    '6-9 kali per bulan',
    '>10 kali per bulan'
]

# Tandai pengguna aktif
pengguna_mb = df_mb[mobile_col].isin(aktif_keywords)

# Hitung jumlah pengguna aktif dan total
count_mb = pengguna_mb.sum()
n_mb = len(df_mb)

print(f"Jumlah data responden (tidak kosong): {n_mb}")
print(f"Jumlah responden yang menggunakan Mobile Banking: {count_mb}")

# Uji hipotesis: proporsi pengguna aktif > 50%
if n_mb == 0 or count_mb == 0:
    print("Data tidak cukup untuk melakukan uji proporsi.")
else:
    stat_mb, pval_mb = proportions_ztest(count_mb, n_mb, value=0.5, alternative='larger')
    print("\n=== One-Sample Z-Test: Apakah pengguna aktif Mobile Banking > 50% ===")
    print(f"Sample proportion: {count_mb}/{n_mb} = {count_mb/n_mb:.3f}")
    print(f"Z-statistic: {stat_mb:.3f}, p-value: {pval_mb:.4f}")
    
    if pval_mb < 0.05:
        print(">> Tolak H0: Proporsi pengguna aktif Mobile Banking > 50% (signifikan)")
    else:
        print(">> Gagal tolak H0: Tidak cukup bukti proporsi > 50%")


Jumlah data responden (tidak kosong): 1000
Jumlah responden yang menggunakan Mobile Banking: 948

=== One-Sample Z-Test: Apakah pengguna aktif Mobile Banking > 50% ===
Sample proportion: 948/1000 = 0.948
Z-statistic: 63.808, p-value: 0.0000
>> Tolak H0: Proporsi pengguna aktif Mobile Banking > 50% (signifikan)


One-sample z-test for proportions:: Apakah proporsi responden yang pernah meminjam online < 40%?

In [17]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

# Load data
df = pd.read_excel("Dataset_Tubes.xlsx")

# ===================== Contoh 2: Proporsi Pinjam Online < 40% =====================
pinjaman_col = 'Apakah_Anda_pernah_mengajukan_pinjaman_online_berikut'
df_pinjaman = df.dropna(subset=[pinjaman_col]).copy()
df_pinjaman[pinjaman_col] = df_pinjaman[pinjaman_col].str.strip().str.lower()

# Anggap semua jawaban selain "tidak pernah menggunakan" sebagai pernah meminjam
pinjam = df_pinjaman[pinjaman_col] != 'tidak pernah menggunakan'

count_pinjaman = pinjam.sum()
n_pinjaman = len(df_pinjaman)

print(f"Jumlah data responden (tidak kosong): {n_pinjaman}")
print(f"Jumlah responden yang pernah melakukan pinjaman online: {count_pinjaman}")

# Z-test: apakah proporsi < 0.4
if n_pinjaman == 0 or count_pinjaman == 0:
    print("Data tidak cukup untuk melakukan uji proporsi.")
else:
    stat_pj, pval_pj = proportions_ztest(count_pinjaman, n_pinjaman, value=0.4, alternative='smaller')
    print("\n=== One-Sample Z-Test: Proporsi Pinjam Online < 40% ===")
    print(f"Sample proportion: {count_pinjaman}/{n_pinjaman} = {count_pinjaman/n_pinjaman:.3f}")
    print(f"Z-statistic: {stat_pj:.3f}, p-value: {pval_pj:.4f}")
    if pval_pj < 0.05:
        print(">> Tolak H0: Proporsi pinjam online < 40% (signifikan pada 5%)")
    else:
        print(">> Gagal tolak H0: Tidak cukup bukti proporsi < 40%")

Jumlah data responden (tidak kosong): 1000
Jumlah responden yang pernah melakukan pinjaman online: 94

=== One-Sample Z-Test: Proporsi Pinjam Online < 40% ===
Sample proportion: 94/1000 = 0.094
Z-statistic: -33.158, p-value: 0.0000
>> Tolak H0: Proporsi pinjam online < 40% (signifikan pada 5%)


Two-sample z-test for differences in proportions:  Apakah proporsi pengguna Mobile Banking berbeda antara laki-laki dan perempuan?

In [31]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

# Load data
df = pd.read_excel("Dataset_Tubes.xlsx")

# Normalisasi kolom
df.columns = df.columns.str.strip().str.replace('\n', '_').str.replace(' ', '_')
df = df.rename(columns={
    'Jenis_kelamin': 'Jenis_Kelamin',
    'Bagaimana_frekuensi_penggunaan_layanan_layanan_Bank_berikut___Mobile_Banking': 'Mobile_Banking',
    'Apakah_Anda_pernah_mengajukan_pinjaman_online_berikut': 'Pinjam_Online',
    'Usia__dalam_tahun': 'Usia'
})
df['Jenis_Kelamin'] = df['Jenis_Kelamin'].str.strip().str.lower()
df['Mobile_Banking'] = df['Mobile_Banking'].astype(str).str.strip().str.lower()
df['Pinjam_Online'] = df['Pinjam_Online'].astype(str).str.strip().str.lower()

# Anggap pengguna aktif: menjawab 'sering', 'selalu', atau mengandung 'kali'
aktif_mb = df['Mobile_Banking'].str.contains('2-5|6-9|>10|sering|selalu')

# Buat kelompok gender
male = df[df['Jenis_Kelamin'] == 'laki-laki']
female = df[df['Jenis_Kelamin'] == 'perempuan']

x1 = aktif_mb[male.index].sum()
n1 = len(male)

x2 = aktif_mb[female.index].sum()
n2 = len(female)

print(f"Jumlah responden Laki-laki: {n1}")
print(f"Jumlah Laki-laki aktif Mobile Banking: {x1}")
print(f"Jumlah responden Perempuan: {n2}")
print(f"Jumlah Perempuan aktif Mobile Banking: {x2}")

stat1, pval1 = proportions_ztest([x1, x2], [n1, n2], alternative='two-sided')

print("\n=== Pengguna Aktif Mobile Banking (Laki-laki vs Perempuan) ===")
print(f"Laki-laki: {x1}/{n1} = {x1/n1:.2f}")
print(f"Perempuan: {x2}/{n2} = {x2/n2:.2f}")
print(f"Z-stat: {stat1:.3f}, p-value: {pval1:.4f}")
if pval1 < 0.05:
    print(">> Ada perbedaan signifikan antara gender dalam penggunaan Mobile Banking.")
else:
    print(">> Tidak ada bukti perbedaan signifikan.")

Jumlah responden Laki-laki: 456
Jumlah Laki-laki aktif Mobile Banking: 428
Jumlah responden Perempuan: 544
Jumlah Perempuan aktif Mobile Banking: 520

=== Pengguna Aktif Mobile Banking (Laki-laki vs Perempuan) ===
Laki-laki: 428/456 = 0.94
Perempuan: 520/544 = 0.96
Z-stat: -1.226, p-value: 0.2201
>> Tidak ada bukti perbedaan signifikan.


Two-sample z-test for differences in proportions:  Apakah proporsi peminjam online lebih tinggi di kelompok usia < 35 tahun dibanding ≥ 35 tahun?

In [30]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

# Load data
df = pd.read_excel("Dataset_Tubes.xlsx")

# Normalisasi kolom jika belum
if 'Pinjam_Online' not in df.columns:
    df = df.rename(columns={'Apakah_Anda_pernah_mengajukan_pinjaman_online_berikut': 'Pinjam_Online'})
df['Pinjam_Online'] = df['Pinjam_Online'].astype(str).str.strip().str.lower()

# Bagi usia menjadi 2 kelompok
df['Kelompok_Usia'] = df['Usia__dalam_tahun'].apply(lambda x: '<35' if x < 35 else '≥35')
group1 = df[df['Kelompok_Usia'] == '<35']
group2 = df[df['Kelompok_Usia'] == '≥35']

# Jawaban selain "tidak pernah menggunakan" dianggap pernah pinjam
x1 = (group1['Pinjam_Online'] != 'tidak pernah menggunakan').sum()
n1 = len(group1)

x2 = (group2['Pinjam_Online'] != 'tidak pernah menggunakan').sum()
n2 = len(group2)

print(f"Jumlah responden <35 tahun: {n1}")
print(f"Jumlah pernah pinjam online <35 tahun: {x1}")
print(f"Jumlah responden ≥35 tahun: {n2}")
print(f"Jumlah pernah pinjam online ≥35 tahun: {x2}")

stat2, pval2 = proportions_ztest([x1, x2], [n1, n2], alternative='larger')

print("\n=== Proporsi Pinjam Online (<35 vs ≥35 Tahun) ===")
print(f"<35 tahun: {x1}/{n1} = {x1/n1:.2f}")
print(f"≥35 tahun: {x2}/{n2} = {x2/n2:.2f}")
print(f"Z-stat: {stat2:.3f}, p-value: {pval2:.4f}")
if pval2 < 0.05:
    print(">> Proporsi kelompok <35 tahun lebih tinggi secara signifikan.")
else:
    print(">> Tidak ada bukti proporsi kelompok <35 tahun lebih tinggi.")

Jumlah responden <35 tahun: 753
Jumlah pernah pinjam online <35 tahun: 73
Jumlah responden ≥35 tahun: 247
Jumlah pernah pinjam online ≥35 tahun: 21

=== Proporsi Pinjam Online (<35 vs ≥35 Tahun) ===
<35 tahun: 73/753 = 0.10
≥35 tahun: 21/247 = 0.09
Z-stat: 0.557, p-value: 0.2887
>> Tidak ada bukti proporsi kelompok <35 tahun lebih tinggi.
